In [1]:
import torch, numpy

In [ ]:
import torch, math
from torch.autograd import Function
class MyMul(Function):
    @staticmethod
    def forward(ctx, x, y):
        ctx.save_for_backward(x,y)
        return x * y

    @staticmethod
    def backward(ctx, grad_output):
        x,y = ctx.saved_tensors
        return grad_output * y,grad_output * x

class MyMax(Function):
    @staticmethod
    def forward(ctx, x, y):
        ctx.save_for_backward(x,y)
        return torch.maximum(x,y)
    
    @staticmethod
    def backward(ctx, grad_output):
        x,y  = ctx.saved_tensors
        if x > y:
            return grad_output, 0
        elif y > x:
            return 0, grad_output 
        else:
            return grad_output * 0.5, grad_output * 0.5